In [1]:
from numpy import array
from numpy import argmax
import numpy as np
import pandas as pd
import glob

dirty_sentences = list()

files = glob.glob("carroll\\*.txt")

############### sentence split ###############
for file_ in files:
	#utf8 doen't work, utf-8-sig removes BOM ufeff
	#with open(file_, 'r', encoding='utf-8-sig') as f: 
	with open(file_, 'r', encoding='utf-8') as f: 
		dirty_sentences += [line.lower().rstrip('\n') for line in f if line.rstrip('\n')!='']
		#append is twice fast, but my case, this is simple enough.

dirty_sentences = ' '.join(dirty_sentences)
############### sentence split ###############

import nltk.data
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
documents = punkt_sents = sent_detector.tokenize(dirty_sentences.strip())

#print(punkt_sents[:20])

############### tokenize ###############
#from nltk.parse.stanford import StanfordParser
#parser=StanfordParser(model_path="D:\GU_MLT\Semester_3\Machine_Learning\lab2\stanford-parser-full-2017-06-09\englishPCFG.ser.gz")
#list(parser.raw_parse("the quick brown fox jumps over the lazy dog"))

# lines = ['line1', 'line2']
# with open('filename.txt', 'w') as f:
#     f.writelines("%s\n" % l for l in lines)

with open('documents_sents.txt', 'w', encoding='utf8') as f:
	f.writelines("%s\n" % l for l in documents)

from nltk.tokenize.stanford import StanfordTokenizer
#from nltk.tokenize.stanford import CoreNLPTokenizer
# StanfordTokenizer().tokenize(s)

tokenized_sents = list()
i=1
total = len(punkt_sents)



In [ ]:
#import pickle
#pickle.dump(tokenized_sents, open('tokenized_sents.p', 'wb'))

In [2]:
import pickle
tokenized_sents = pickle.load(open('tokenized_sents.p', 'rb'))

In [3]:
flatten_voca = [a for b in tokenized_sents for a in b]
print("flatten_voca length is ", len(flatten_voca))
vocabulary = sorted(list(set(flatten_voca)))

with open('vocabulary.txt', 'w', encoding='utf8') as f:
	f.writelines("%s\n" % l for l in vocabulary)


print(tokenized_sents[:5])
############### trigram by n-gram from nltk ###############
#tokenized_sents
from nltk import ngrams
from sklearn import preprocessing
#from sklearn.preprocessing import OneHotEncoder

# le  = preprocessing.LabelEncoder()
ohe = preprocessing.OneHotEncoder() 
#one-hot representations

# le.fit(vocabulary)#

onehot = ohe.fit_transform([[i] for i in range(len(vocabulary))]).toarray()
#one line : onehot = np_utils.to_categorical(ohe.fit_transform(vocabulary))
"""
array(word1	   [[ 1.,  0.,  0., ...,  0.,  0.,  0.],
	  word2  	[ 0.,  1.,  0., ...,  0.,  0.,  0.],
      word3 	[ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
      word[N-2] [ 0.,  0.,  0., ...,  1.,  0.,  0.],
      word[N-1] [ 0.,  0.,  0., ...,  0.,  1.,  0.],
      word[N] 	[ 0.,  0.,  0., ...,  0.,  0.,  1.]])
"""


flatten_voca length is  69747
[['<Start>', 'project', 'gutenberg', "'s", 'alice', "'s", 'adventures', 'under', 'ground', ',', 'by', 'lewis', 'carroll', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '.', '<End>'], ['<Start>', 'you', 'may', 'copy', 'it', ',', 'give', 'it', 'away', 'or', 're-use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www.gutenberg.org', 'title', ':', 'alice', "'s", 'adventures', 'under', 'ground', 'author', ':', 'lewis', 'carroll', 'release', 'date', ':', 'august', '7', ',', '2006', '-LSB-', 'ebook', '#', '19002', '-RSB-', 'language', ':', 'english', '***', 'start', 'of', 'this', 'project', 'gutenberg', 'ebook', 'alice', "'s", 'adventures', 'under', 'ground', '***', 'produced', 'by', 'jason', 'isbell', ',', 'sankar', 'viswanathan', ',', 'and', 'the', 'online', 'distr

'\narray(word1\t   [[ 1.,  0.,  0., ...,  0.,  0.,  0.],\n\t  word2  \t[ 0.,  1.,  0., ...,  0.,  0.,  0.],\n      word3 \t[ 0.,  0.,  1., ...,  0.,  0.,  0.],\n       ..., \n      word[N-2] [ 0.,  0.,  0., ...,  1.,  0.,  0.],\n      word[N-1] [ 0.,  0.,  0., ...,  0.,  1.,  0.],\n      word[N] \t[ 0.,  0.,  0., ...,  0.,  0.,  1.]])\n'

In [4]:
onehot = pd.DataFrame(onehot, index=vocabulary)

#print(onehot.shape)


# N-gram
onehot_sent  = list()
onehot_sents = list()
#i =0
#print(tokenized_sents[1], len(tokenized_sents[1]))
for sent in tokenized_sents:
    for word in sent:
        onehot_sent.append(onehot.loc[word])
        i+=1
    #print(len(onehot_sent), len(onehot_sent[0]))
    onehot_sents+=list(ngrams(onehot_sent, 3)) # each list has (onehot_word1, onehot_word2, onehot_word3)
    onehot_sent.clear()
    #print("i is ", i, len(onehot_sents), len(onehot_sents[0]), len(onehot_sents[1]))

onehot_sents = pd.DataFrame(onehot_sents)

In [5]:
onehot_sents.head()

,0,1,2
0,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
1,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
2,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
3,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
4,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....


In [6]:
onehot_sents.iloc[0,0].shape

(4262,)

In [7]:
# import gc
# inputs, outputs = list(), list()
# row_numb, col_numb = onehot_sents.shape
# i = 1

# for index, row in onehot_sents.iterrows():
#     inputs.append((list(row[0]), list(row[1])))
#     outputs.append(list(row[2]))
#     if index == int(row_numb*i/100):
#         print(i, "% is done")
#         i+=1
#         pickle.dump(tokenized_sents, open('inputs'+str(i)+'.p', 'wb'))
#     del [index, row]
#     gc.collect()

# print(len(inputs), len(outputs))
# print(len(inputs[0][0]),type(inputs[0][0]))

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM

Using TensorFlow backend.


In [8]:
len(vocabulary)

4262

In [6]:
# Input layer is dense with the input dimension of two characters side-by-side in 
# our 103-character representation. But we compress to 128. 
# We have sigmoid activation and 10% dropout.
# We get a 103-wide output layer to get the third character, and a softmax layer
# for the actual prediction.
model = Sequential()

# model.add(LSTM(256, input_shape=(1, 4262*2), return_sequences=True))
# model.add(Activation('sigmoid'))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Activation('sigmoid'))
# model.add(Dropout(0.2))
# model.add(Dense(units=4262))
# model.add(Activation('softmax'))


model.add(Dense(units=256, input_dim=4262*2))
model.add(Activation('sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(units=256, input_dim=4262*2))
model.add(Activation('sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(units=4262))
model.add(Activation('softmax'))

In [7]:
# ADAM is a fancier gradient descent optimizer. The average cross-entropy gives us a
# perplexity measure.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               2182400   
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4262)              1095334   
__________

In [9]:
inout_list = zip(glob.glob("inputs*.p"), glob.glob("outputs*.p"))
inout_list= list(inout_list)
inout_list[:10]

[('inputs1.p', 'outputs1.p'),
 ('inputs10.p', 'outputs10.p'),
 ('inputs11.p', 'outputs11.p'),
 ('inputs12.p', 'outputs12.p'),
 ('inputs13.p', 'outputs13.p'),
 ('inputs14.p', 'outputs14.p'),
 ('inputs15.p', 'outputs15.p'),
 ('inputs16.p', 'outputs16.p'),
 ('inputs17.p', 'outputs17.p'),
 ('inputs18.p', 'outputs18.p')]

In [10]:
import gc, time
inputs = []
outputs = []
temp = []

time_start = time.time()

# 40% inout_list will chock, i5-2.8GHz, 8GB RAM. Gtx840M
for input_file, output_file in inout_list[:75]:
    temp = pickle.load(open(input_file, 'rb'))
    inputs += temp
    temp.clear(); gc.collect()
    temp = pickle.load(open(output_file, 'rb'))
    outputs += temp
    temp.clear(); gc.collect()

print("cost time : %s second" %(time.time() - time_start))

cost time : 81.22364568710327 second


In [11]:
len(inputs), len(outputs) #(13054, 13054)
len(inputs[0]), len(outputs[0]) # (2, 4262)
len(inputs[0][0]), len(outputs[0]) # (4262, 4262)

(4262, 4262)

In [12]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='weights80percent_e50_b16.hdf5', verbose=1, save_best_only=True)

In [13]:
time_start = time.time()
longinputs = [list(i[0].as_matrix())+list(i[1].as_matrix()) for i in inputs]
print("cost time : {} second".format(time.time() - time_start))
#cost time : 372.3062946796417 second

cost time : 156.17893290519714 second


In [14]:
time_start = time.time()
history = model.fit(np.array(longinputs), np.array(outputs), epochs=50, batch_size=16, validation_split=0.1, callbacks=[checkpointer])
print("cost time : {} second".format(time.time() - time_start))

import h5py
model.save('carroll_model80.h5')
history.params

# 35% data
# Epoch 50/50
# 20544/20559 [============================>.] - ETA: 0s - loss: 2.2758 - acc: 0.4596Epoch 00049: val_loss did not improve
# 20559/20559 [==============================] - 68s - loss: 2.2760 - acc: 0.4595 - val_loss: 6.5106 - val_acc: 0.1589
# cost time : 3561.255692243576 second

# 80% data
# Epoch 50/50
# 44048/44055 [============================>.] - ETA: 0s - loss: 2.4035 - acc: 0.4180Epoch 00049: val_loss did not improve
# 44055/44055 [==============================] - 138s - loss: 2.4034 - acc: 0.4180 - val_loss: 5.9511 - val_acc: 0.1785
# cost time : 7536.25204873085 second

Train on 44055 samples, validate on 4895 samples
Epoch 1/50
44055/44055 [==============================] - 225s - loss: 5.9249 - acc: 0.0867 - val_loss: 5.6617 - val_acc: 0.0981
Epoch 2/50
44055/44055 [==============================] - 152s - loss: 5.5630 - acc: 0.1086 - val_loss: 5.5831 - val_acc: 0.1101
Epoch 3/50
44055/44055 [==============================] - 140s - loss: 5.3800 - acc: 0.1205 - val_loss: 5.5321 - val_acc: 0.1236
Epoch 4/50
44055/44055 [==============================] - 140s - loss: 5.2338 - acc: 0.1326 - val_loss: 5.5235 - val_acc: 0.1406
Epoch 5/50
44055/44055 [==============================] - 140s - loss: 5.0793 - acc: 0.1499 - val_loss: 5.4093 - val_acc: 0.1504
Epoch 6/50
44055/44055 [==============================] - 140s - loss: 4.9226 - acc: 0.1653 - val_loss: 5.3241 - val_acc: 0.1810
Epoch 7/50
44055/44055 [==============================] - 139s - loss: 4.7794 - acc: 0.1805 - val_loss: 5.2483 - val_acc: 0.1783
Epoch 8/50
44055/44055 [========================

44055/44055 [==============================] - 136s - loss: 3.2317 - acc: 0.3177 - val_loss: 5.3893 - val_acc: 0.2061
Epoch 31/50
44055/44055 [==============================] - 137s - loss: 3.1777 - acc: 0.3237 - val_loss: 5.4415 - val_acc: 0.1992
Epoch 32/50
44055/44055 [==============================] - 137s - loss: 3.1319 - acc: 0.3276 - val_loss: 5.4760 - val_acc: 0.2012
Epoch 33/50
44055/44055 [==============================] - 137s - loss: 3.0853 - acc: 0.3303 - val_loss: 5.5061 - val_acc: 0.1965
Epoch 34/50
44055/44055 [==============================] - 138s - loss: 3.0341 - acc: 0.3359 - val_loss: 5.5193 - val_acc: 0.1971
Epoch 35/50
44055/44055 [==============================] - 138s - loss: 2.9884 - acc: 0.3406 - val_loss: 5.5507 - val_acc: 0.1992
Epoch 36/50
44055/44055 [==============================] - 138s - loss: 2.9437 - acc: 0.3470 - val_loss: 5.5899 - val_acc: 0.1926
Epoch 37/50
44055/44055 [==============================] - 138s - loss: 2.8939 - acc: 0.3532 - val_los

{'batch_size': 16,
 'do_validation': True,
 'epochs': 50,
 'metrics': ['loss', 'acc', 'val_loss', 'val_acc'],
 'samples': 44055,
 'steps': None,
 'verbose': 1}

In [ ]:
history

In [15]:
import h5py
model.save('carroll_model80.h5')
history.params

{'batch_size': 16,
 'do_validation': True,
 'epochs': 50,
 'metrics': ['loss', 'acc', 'val_loss', 'val_acc'],
 'samples': 44055,
 'steps': None,
 'verbose': 1}

In [5]:
from keras.models import load_model
model = load_model('weights.hdf5')

In [16]:
inputs  = pickle.load(open("inputs99.p", 'rb'))
outputs = pickle.load(open("outputs99.p", 'rb'))

longinputs = [list(i[0].as_matrix())+list(i[1].as_matrix()) for i in inputs] #should be, (653, 8524)

model.evaluate(np.array(longinputs), np.array(outputs))
# before : [8.3058816287638031, 0.099540581929555894]
# after  : [8.4662098921093794, 0.10107197549770292]
# weight3 : [8.4188709536518846, 0.05359877488514548]
# weights3_e50_b16 : [1.5340424204675929, 0.62327718296605994]
# weights80percent_e50_b16 : [1.6102490667909795, 0.61715160942369729 

608/653 [==========================>...] - ETA: 0s

[1.6102490667909795, 0.61715160942369729]

In [17]:
def bigram(word1, word2):
    index1 = vocabulary.index(word1)
    index2 = vocabulary.index(word2)
    vec1 = [0] * 4262 #len(vocabulary)
    vec2 = [0] * 4262
    vec1[index1] = 1
    vec2[index2] = 1
    return vec1 + vec2

In [18]:
the_book_bigram = bigram("<Start>", "alice")
the_book_bigram2 = bigram("<Start>", "king")

print(np.argmax(the_book_bigram), sum(the_book_bigram))
print(np.argmax(the_book_bigram2), sum(the_book_bigram2), len(the_book_bigram2))

print(vocabulary.index("<Start>"), vocabulary.index("alice"), vocabulary.index("i"))
ve1, ve2, ve3 = [0]*4262, [0]*4262, [0]*4262 
ve1[94], ve2[582], ve3[1912] = 1, 1, 1

print(the_book_bigram == ve1 + ve2)
print(the_book_bigram2 == ve1 + ve3)

for n, v in enumerate(the_book_bigram2):
    if v!=0 : print(n, v)

94 2
94 2 8524
94 188 1912
False
False
94 1
6328 1


In [19]:
import numpy as np
the_book_prediction = model.predict(np.array([the_book_bigram]))
the_book_prediction2 = model.predict(np.array([the_book_bigram2]))

In [20]:
N      = 20
skip_N = 20

next_word    = np.array(the_book_prediction)
next_word2    = np.array(the_book_prediction2)

#print(next_word[0])
argmax_top_N = next_word.argsort()[0][-(skip_N +N):-skip_N][::-1]
argmax_top_N2 = next_word2.argsort()[0][-N:][::-1]

print(argmax_top_N)
print(argmax_top_N2)
print(set(argmax_top_N)==set(argmax_top_N2))

[1502   20  282 3176  385 4060 3249 1763 3474 2449  582 4090 2009 3190  191
  581  214 3486 4122 1595]
[  20 4085  214 3315   14 1018 3176 4130  275  188 3721 2249 1775 2009    0
 3564 3727 3731 1815  332]
False


In [21]:
candidate_list = [vocabulary[next_word] for next_word in argmax_top_N]
for next_word in argmax_top_N :
    print(vocabulary[next_word])

folded
,
asked
said
began
waited
seemed
hastily
soon
must
can
watched
is
sat
all
came
and
sounded
were
gave


In [24]:
N      = 7
skip_N = 2

LM_sentence = ["<Start>"]

next_word = np.random.choice(vocabulary)
LM_sentence.append(next_word)
#LM_sentence.append("alice")
find_END  = False
duplicate = False

while not find_END:

    if LM_sentence[-1] == "<End>" :
        break; find_END=True;
    try : the_book_bigram = bigram(LM_sentence[-2], LM_sentence[-1])
    # some words don't exsit in index, e.g., 2009 is not in list
    except : continue
    
    the_book_prediction = model.predict(np.array([the_book_bigram]))
    next_word    = np.array(the_book_prediction)
    #argmax_top_N = next_word.argsort()[0][-N:][::-1]
    argmax_top_N = next_word.argsort()[0][-(skip_N +N):-skip_N][::-1]
    
    candidate_list = [vocabulary[next_word] for next_word in argmax_top_N]
    
    next_word = np.random.choice(candidate_list)
    
    #want to not save -RRB-, -LRB- stuff.
    if next_word.startswith('-') and next_word.endswith('-') : continue
    elif next_word in LM_sentence : continue
    
    LM_sentence.append(next_word)
    
    print(next_word)
    if next_word == "<End>" :
        find_END = True

print(LM_sentence)

and
whispered
.
''
but
a
great
copy
of
it
!
_
each
project
works
's
hands
that
there
king
did
his
manage
the
table
behind
another
:
then
they
were
comfortably
mostly
poems
<End>
['<Start>', 'soldiers', 'and', 'whispered', '.', "''", 'but', 'a', 'great', 'copy', 'of', 'it', '!', '_', 'each', 'project', 'works', "'s", 'hands', 'that', 'there', 'king', 'did', 'his', 'manage', 'the', 'table', 'behind', 'another', ':', 'then', 'they', 'were', 'comfortably', 'mostly', 'poems', '<End>']
